# Importing packages

In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
from gtsimulation.Algos import BunemanBorisSimulator
from gtsimulation.Global import Regions, Constants, Units
from gtsimulation.MagneticFields import Uniform
from gtsimulation.Particle import Generators, Flux

# General GT settings

In [ ]:
B = 0.1  # magnetic field [T]
T = 100  # energy [MeV]
v_0 = np.array([0, -5, 1]) / np.sqrt(26)  # initial unit velocity

In [ ]:
date = datetime(2008, 1, 1)
region = Regions.Undefined
b_field = Uniform(B=np.array([0, 0, B * 1e9]))
medium = None

use_decay = False
nuclear_interaction = None

dt = 1e-9  # time step [s]
n_steps = 3000
break_conditions = None

save = [1, {"Coordinates": True, "Velocities": True}]
output = None

verbose = True

# Forward tracing

In [ ]:
particle_forward = Flux(
    Spectrum=Generators.Spectrums.Monolines(energy=T * Units.MeV),
    Distribution=Generators.Distributions.UserInput(
        R0=[14.542, 0, 0],
        V0=v_0
    ),
    Names="proton",
    Nevents=1
)

In [ ]:
simulator_forward = BunemanBorisSimulator(
    Bfield=b_field,
    Region=region,
    Medium=medium,
    Particles=particle_forward,
    InteractNUC=nuclear_interaction,
    UseDecay=use_decay,
    Date=date,
    Step=dt,
    Num=n_steps,
    ForwardTrck=1,
    BreakCondition=break_conditions,
    Save=save,
    Output=output,
    Verbose=verbose
)
track_forward = simulator_forward()[0][0]

In [ ]:
r_forward = track_forward["Track"]["Coordinates"]
v_forward = track_forward["Track"]["Velocities"]

# Backward tracing

In [ ]:
particle_backward = Flux(
    Spectrum=Generators.Spectrums.Monolines(energy=T * Units.MeV),
    Distribution=Generators.Distributions.UserInput(
        R0=r_forward[-1, :],
        V0=v_forward[-1, :]
    ),
    Names="proton",
    Nevents=1
)

In [ ]:
simulator_backward = BunemanBorisSimulator(
    Bfield=b_field,
    Region=region,
    Medium=medium,
    Particles=particle_backward,
    InteractNUC=nuclear_interaction,
    UseDecay=use_decay,
    Date=date,
    Step=dt,
    Num=n_steps,
    ForwardTrck=-1,
    BreakCondition=break_conditions,
    Save=save,
    Output=output,
    Verbose=verbose
)
track_backward = simulator_backward()[0][0]

In [ ]:
r_backward = track_backward["Track"]["Coordinates"]

# Analytical solution

In [ ]:
m_p = 938.27  # proton mass [MeV/c²]
gamma = (m_p + T) / m_p
omega = Constants.e * B / (gamma * m_p * Units.MeV2kg)  # cyclotron frequency [Hz]

_, v_y, v_z = Constants.c * np.sqrt(1 - 1 / gamma ** 2) * v_0
r_larmor = gamma * m_p * Units.MeV2kg * np.abs(v_y) / (Constants.e * B)

t = np.linspace(0, dt * n_steps, n_steps)
x = r_larmor * np.cos(omega * t)
y = -r_larmor * np.sin(omega * t)
z = v_z * t

r_analytic = np.stack((x, y, z), axis=1)

# Comparison

In [ ]:
fig = plt.figure(figsize=(12, 6))

ax3d = fig.add_subplot(1, 2, 1, projection='3d')

ax3d.plot(*r_forward.T, label="Forward")
ax3d.plot(*r_backward.T, label="Backward", linestyle="--")
ax3d.plot(*r_analytic.T, label="Analytic", linestyle=":")

ax3d.scatter(*r_forward[0], label='Proton initial position')
ax3d.scatter(*r_backward[0], label="Antiproton initial position")

ax3d.set_xlabel("X [m]")
ax3d.set_ylabel("Y [m]")
ax3d.set_zlabel("Z [m]")
ax3d.set_aspect('equalxy')
ax3d.legend()

ax2d = fig.add_subplot(1, 2, 2)

ax2d.plot(*r_forward.T[:2], label="Forward")
ax2d.plot(*r_backward.T[:2], label="Backward", linestyle="--")
ax2d.plot(*r_analytic.T[:2], label="Analytic", linestyle=":")

ax2d.scatter(*r_forward[0][:2])
ax2d.scatter(*r_backward[0][:2])

ax2d.set_xlabel("X [m]")
ax2d.set_ylabel("Y [m]")
ax2d.set_aspect('equal')
ax2d.grid(True, linestyle='--', alpha=0.8)

fig.subplots_adjust(wspace=0.3)

plt.show()

# Deviation from the analytical solution

In [ ]:
# Forward trajectory
dot_forward = np.sum(r_forward * r_analytic, axis=1)
norm_forward = np.linalg.norm(r_forward, axis=1) * np.linalg.norm(r_analytic, axis=1)
phase_forward = np.rad2deg(np.arccos(dot_forward / norm_forward))

# Backward trajectory
dot_backward = np.sum(r_backward * r_analytic[::-1], axis=1)
norm_backward = np.linalg.norm(r_backward, axis=1) * np.linalg.norm(r_analytic[::-1], axis=1)
phase_backward = np.rad2deg(np.arccos(dot_backward / norm_backward))

In [ ]:
fig = plt.figure()
ax = fig.subplots()

ax.plot(phase_forward, label="Forward")
ax.plot(phase_backward, label="Backward")

ax.set_xlabel("Number of steps")
ax.set_ylabel(r"$\theta$ [deg]")
ax.legend()

plt.show()